#Data Preprocessing

In [1]:
import pandas as pd

masses_data = pd.read_csv('/content/mammographic_masses.data.txt')
masses_data.head()

,5,67,3,5.1,3.1,1
0,4,43,1,1,?,1
1,5,58,4,5,3,1
2,4,28,1,1,3,0
3,5,74,1,5,?,1
4,4,65,1,?,3,0


In [2]:
masses_data = pd.read_csv('/content/mammographic_masses.data.txt', na_values=['?'], names = ['BI-RADS', 'age', 'shape', 'margin', 'density', 'severity'])
masses_data.head()

,BI-RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1


In [3]:
masses_data.describe()

,BI-RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


In [4]:
masses_data.loc[(masses_data['age'].isnull()) |
              (masses_data['shape'].isnull()) |
              (masses_data['margin'].isnull()) |
              (masses_data['density'].isnull())]

,BI-RADS,age,shape,margin,density,severity
1,4.0,43.0,1.0,1.0,NaN,1
4,5.0,74.0,1.0,5.0,NaN,1
5,4.0,65.0,1.0,NaN,3.0,0
6,4.0,70.0,NaN,NaN,3.0,0
7,5.0,42.0,1.0,NaN,3.0,0
...,...,...,...,...,...,...
778,4.0,60.0,NaN,4.0,3.0,0
819,4.0,35.0,3.0,NaN,2.0,0
824,6.0,40.0,NaN,3.0,4.0,1
884,5.0,NaN,4.0,4.0,3.0,1


In [5]:
masses_data.dropna(inplace=True)
masses_data.describe()

,BI-RADS,age,shape,margin,density,severity
count,830.000000,830.000000,830.000000,830.000000,830.000000,830.000000
mean,4.393976,55.781928,2.781928,2.813253,2.915663,0.485542
std,1.888371,14.671782,1.242361,1.567175,0.350936,0.500092
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,46.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


## Extracting Features

In [6]:
all_features = masses_data[['age', 'shape',
                             'margin', 'density']].values


all_classes = masses_data['severity'].values

feature_names = ['age', 'shape', 'margin', 'density']


##Normalizing the data


In [7]:
from sklearn import preprocessing

scaler = preprocessing.StandardScaler()
all_features_scaled = scaler.fit_transform(all_features)
all_features_scaled

array([[ 0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [ 0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [ 0.42406719,  0.17563638,  0.11923341,  0.24046607]])

In [8]:
from tensorflow.keras.utils import to_categorical
classes = to_categorical(all_classes)
classes

array([[0., 1.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

##Train-Test Split

In [9]:
import numpy
from sklearn.model_selection import train_test_split

numpy.random.seed(1234)

(training_inputs,
 testing_inputs,
 training_classes,
 testing_classes) = train_test_split(all_features_scaled, classes, train_size=0.75, random_state=1)

In [10]:
import numpy as np
from numpy import exp
np.random.seed(0)


##Class representing a linear layer having n_inp neurons as input and n_out neurons as output

In [11]:
class Linearlayer():
    def __init__(self,n_inp,n_out):
        self.weights = np.random.randn(n_inp,n_out)
        self.bias = np.zeros((1,n_out))
    def forward(self,inputs):
        self.output = np.dot(inputs,self.weights) + self.bias


##Neural Network

In [12]:
class NeuralNet():
    def __init__(self,n_inp,n_out,alpha):
        self.inp = n_inp
        self.out = n_out
        self.hidd_no1 = 60
        self.hidd_no2 = 80
        self.alpha = alpha
        self.error = 1
        self.layer1 = Linearlayer(n_inp,self.hidd_no1)
        self.layer2 = Linearlayer(self.hidd_no1,self.hidd_no2)
        self.layer3 = Linearlayer(self.hidd_no2,n_out)
        
    # sigmoid activation function
    def act_fun(self,x):
        return 1/(1+exp(-x))
    # derivation of sigmoid function
    def der_act_fun(self,x):
        x = self.act_fun(x)
        return x*(1-x)
    # forward pass through the network
    def forward(self,input_set):
        self.input_set = input_set
        self.layer1.forward(input_set)
        # applying activation function after layer1 pass
        self.inp_hidden1 = self.act_fun(self.layer1.output)
        # applying activation function after layer2 pass
        self.layer2.forward(self.inp_hidden1)
        self.inp_hidden2 = self.act_fun(self.layer2.output)
        self.layer3.forward(self.inp_hidden2)
        # applying activation function after layer3 pass
        self.fout = self.act_fun(self.layer3.output)
        return self.fout
    # This is the main function from which neural network will learn
    def learn(self,input_set,output_set):
        nnout = self.forward(input_set)
#         print("output is - ",nnout)
        self.error = 0
        for i in range(len(output_set)):
            self.error+=(output_set[i]-nnout[0][i])**2
        self.error/=2
        # print("error - ",self.error)


        # Backpropogating the error
        self.backpropgatel1(output_set)
        self.backpropgatel2()
        self.backpropgatel3()

      # backpropogate through layer1
    def backpropgatel1(self,output):
        self.errorhid1 = []
        xins = self.inp_hidden2[0]
        yout = self.fout[0]
        yins = self.layer3.output[0]
        for i in range(len(xins)):
            for j in range(len(yout)):
                diff = -1*(output[j]-yout[j])*self.der_act_fun(yins[j])
                chw = diff*xins[i]
#                 print("chw at ",i,j," is",chw)
                self.errorhid1.append(diff)
                self.layer3.weights[i][j]-= self.alpha*chw
        for j in range(len(yout)):
            self.layer3.bias[0][j]-=self.alpha*self.errorhid1[j]
      
      
      # backpropogate through layer2
    def backpropgatel2(self):
        self.errorhid2 = []
        for i in range(self.hidd_no1):
            for j in range(self.hidd_no2):
                cng = 0
                for k in range(len(self.layer3.weights[j])):
                    cng += self.errorhid1[k]*self.layer3.weights[j][k]
                diff = cng*self.der_act_fun(self.layer2.output[0][j])
                self.errorhid2.append(diff)
                cng=diff*self.inp_hidden1[0][i]
                self.layer2.weights[i][j]-=self.alpha*cng
        
        for j in range(self.hidd_no2):
            self.layer2.bias[0][j]-=self.alpha*self.errorhid2[j]
            


    # backpropogate through layer3
    def backpropgatel3(self):
        self.errorhid3 = []
        for i in range(self.inp):
            for j in range(self.hidd_no1):
                cng = 0
                for k in range(len(self.layer2.weights[j])):
                    cng+=self.errorhid2[k]*self.layer2.weights[j][k]
                diff = cng*self.der_act_fun(self.layer1.output[0][j])
                self.errorhid3.append(diff)
                cng = diff*self.input_set[i]
                self.layer1.weights[i][j]-=self.alpha*cng
            for j in range(self.hidd_no1):
                self.layer1.bias[0][j]-=self.alpha*self.errorhid3[j]


    
    def predict(self , features):
        lst = list(self.forward(features)[0])
        index = lst.index(max(lst))
        return index

    def percentage_classes(self):
      lst = list(self.fout[0])
      per = []
      smx = 0
      for e in lst:
        smx+=exp(-e)
      for e in lst:
        pro = exp(-e)/smx
        per.append(pro)
      print(pro)
      return pro

                
                
        

##Neural Network with learning rate 0.1
here hidden params are fixed and can be changed in the class Neural Network
This is a 3 layer Network

In [13]:
net = NeuralNet(4 , 2 , 0.1)

###Training the neural network

In [14]:
size = training_inputs.shape[0]
error = 0
count = 0
lmt = size//4
epochs = 10
for epoch in range(epochs):
  print("Epoch : " , epoch+1)
  for row,label in zip(training_inputs , training_classes):
    net.learn(row , label)
    error += net.error
    count+=1
    if count%lmt==0:
      print(count/size*100 , '%')
      print("\terror = ",error/count)



Epoch :  1
24.919614147909968 %
	error =  0.37316295267838945
49.839228295819936 %
	error =  0.36008832862347684
74.7588424437299 %
	error =  0.35166936979722285
99.67845659163987 %
	error =  0.34444274012584397
Epoch :  2
124.59807073954985 %
	error =  0.34356471086692353
149.5176848874598 %
	error =  0.3408034052424848
174.43729903536976 %
	error =  0.31785406503720737
199.35691318327974 %
	error =  0.299334635703704
Epoch :  3
224.2765273311897 %
	error =  0.28670935999488634
249.1961414790997 %
	error =  0.2750164797112357
274.11575562700966 %
	error =  0.2597799368104917
299.0353697749196 %
	error =  0.2526591118364677
Epoch :  4
323.95498392282957 %
	error =  0.24635722458252343
348.8745980707395 %
	error =  0.2402873113551671
373.7942122186495 %
	error =  0.2313365837774392
398.7138263665595 %
	error =  0.22776519424364086
Epoch :  5
423.6334405144695 %
	error =  0.2236774271381104
448.5530546623794 %
	error =  0.2206277941285563
473.4726688102894 %
	error =  0.21445080980442455

###Checking the accuracy of the model
learning rate = 0.1


In [15]:
count = 0
correct = 0
for row,label in zip(testing_inputs , testing_classes):
  lst = list(label)
  index = lst.index(max(lst))
  value = net.predict(row)
  count+=1
  if index == value:
    correct+=1
print("Accuracy of the model : " , correct/count*100)

Accuracy of the model :  77.88461538461539


##Neural Network with learning rate 0.01
This is a 3 layer Network

In [16]:
net2 = NeuralNet(4,2,0.01)

###Training the neural Network

In [17]:
size = training_inputs.shape[0]
error = 0
count = 0
lmt = size//4
epochs = 10
for epoch in range(epochs):
  print("Epoch : " , epoch+1)
  for row,label in zip(training_inputs , training_classes):
    net2.learn(row , label)
    error += net2.error
    count+=1
    if count%lmt==0:
      print(count/size*100 , '%')
      print("\terror = ",error/count)



Epoch :  1
24.919614147909968 %
	error =  0.4484474810441062
49.839228295819936 %
	error =  0.4187063397962759
74.7588424437299 %
	error =  0.4089126056414229
99.67845659163987 %
	error =  0.39238299424305456
Epoch :  2
124.59807073954985 %
	error =  0.3845270884470666
149.5176848874598 %
	error =  0.37998819052663796
174.43729903536976 %
	error =  0.375753665353778
199.35691318327974 %
	error =  0.3698458422399756
Epoch :  3
224.2765273311897 %
	error =  0.36697641486607235
249.1961414790997 %
	error =  0.3646538837996347
274.11575562700966 %
	error =  0.3627257993481385
299.0353697749196 %
	error =  0.3598928989569437
Epoch :  4
323.95498392282957 %
	error =  0.3581602126613841
348.8745980707395 %
	error =  0.3565198966310553
373.7942122186495 %
	error =  0.3552361768364237
398.7138263665595 %
	error =  0.35368329028409445
Epoch :  5
423.6334405144695 %
	error =  0.3523283579207624
448.5530546623794 %
	error =  0.3513375499647282
473.4726688102894 %
	error =  0.35060492533137544
498.

###Checking the accuracy of model
learning rate = 0.01

In [18]:
count = 0
correct = 0
for row,label in zip(testing_inputs , testing_classes):
  lst = list(label)
  index = lst.index(max(lst))
  value = net2.predict(row)
  count+=1
  if index == value:
    correct+=1
print("Accuracy of the model : " , correct/count*100)

Accuracy of the model :  57.21153846153846


##Neural Network with learning rate 0.3
This is a 3 layer Network

In [19]:
net3 = NeuralNet(4 , 2 , 0.3)

###Training the network

In [20]:
size = training_inputs.shape[0]
error = 0
count = 0
lmt = size//4
epochs = 10
for epoch in range(epochs):
  print("Epoch : " , epoch+1)
  for row,label in zip(training_inputs , training_classes):
    net3.learn(row , label)
    error += net3.error
    count+=1
    if count%lmt==0:
      print(count/size*100 , '%')
      print("\terror = ",error/count)



Epoch :  1
24.919614147909968 %
	error =  0.36405487692277777
49.839228295819936 %
	error =  0.35555470122309746
74.7588424437299 %
	error =  0.3363132915384887
99.67845659163987 %
	error =  0.34155301291809004
Epoch :  2
124.59807073954985 %
	error =  0.34310272320172075
149.5176848874598 %
	error =  0.3433185674614842
174.43729903536976 %
	error =  0.33568981948658255
199.35691318327974 %
	error =  0.3272192817843536
Epoch :  3
224.2765273311897 %
	error =  0.31213999202962267
249.1961414790997 %
	error =  0.30105245894380755
274.11575562700966 %
	error =  0.2834862590242351
299.0353697749196 %
	error =  0.27657853729931464
Epoch :  4
323.95498392282957 %
	error =  0.27045789778371354
348.8745980707395 %
	error =  0.2648502042051081
373.7942122186495 %
	error =  0.25449642704536923
398.7138263665595 %
	error =  0.2510449965548078
Epoch :  5
423.6334405144695 %
	error =  0.24693345733806735
448.5530546623794 %
	error =  0.24412688179693187
473.4726688102894 %
	error =  0.2369546853796

###Checking the accuracy of the model
learning rate = 0.3

In [22]:
count = 0
correct = 0
for row,label in zip(testing_inputs , testing_classes):
  lst = list(label)
  index = lst.index(max(lst))
  value = net3.predict(row)
  count+=1
  if index == value:
    correct+=1
print("Accuracy of the model : " , correct/count*100)

Accuracy of the model :  77.40384615384616
